In [1]:
# Data processing
import pandas as pd
import numpy as np

# Modeling
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TextClassificationPipeline
from transformers import pipeline

# Hugging Face Dataset
from datasets import Dataset


/Users/johanneswidera/Uni/bachelorarbeit/Code/models/HUGGING_ENV/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("./sentiment_transfer_learning_transformer/")

# Load model
loaded_model = AutoModelForSequenceClassification.from_pretrained("./sentiment_transfer_learning_transformer/")

In [3]:
model = pipeline('sentiment-analysis', model=loaded_model, tokenizer=tokenizer)

# Test

In [ ]:
model.report()

In [4]:
import shap
# explain the model on two sample inputs
s = ['IBM Watson was a scam']
print(model(s)[0])


/Users/johanneswidera/Uni/bachelorarbeit/Code/hugging-face/HUGGING_ENV/lib/python3.9/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/Users/johanneswidera/Uni/bachelorarbeit/Code/hugging-face/HUGGING_ENV/lib/python3.9/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-

{'label': 'LABEL_0', 'score': 0.9154695868492126}


In [5]:
import shap
explainer = shap.Explainer(model) 
shap_values = explainer(s)
shap.plots.text(shap_values[0, :, "LABEL_1"])

In [ ]:
# Playing around with the masker

masker = shap.maskers.Text(tokenizer=r"\W+")
explainer = shap.Explainer(model, masker=masker)
shap_values = explainer(s)
print(model(s))
shap.plots.bar(shap_values[0, :, 'LABEL_0'])
shap.plots.waterfall(shap_values[0, :, 'LABEL_0'])